<a href="https://colab.research.google.com/github/thotran2015/COVID19_Causality/blob/master/data_organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
# !pip install -U plotly
# import plotly.express as px
from urllib.request import urlopen

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import pandas
import time
import io
import json

#You will need to mount the drive to use the relevant datasets in the shared 'Data' folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
NYT_LINK = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
CENSUS_LINK = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv'
CENSUS_ENCODING = 'latin1'

nyt_cases = pd.read_csv(NYT_LINK) 
census_data = pd.read_csv(CENSUS_LINK, encoding=CENSUS_ENCODING)
pop_data = census_data[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','POPESTIMATE2019']]

pop_data["fips"] = pop_data['STATE']*1000 + pop_data['COUNTY'] 
case_pop = pd.merge(nyt_cases, pop_data[['POPESTIMATE2019', 'fips']], on = 'fips')
case_pop = case_pop.rename(columns = {'POPESTIMATE2019':'population'})

all_cases_time = case_pop.set_index(pd.to_datetime(case_pop.date))
del all_cases_time['date']

all_cases_time['population_adjusted_cases'] = all_cases_time['cases']/all_cases_time['population']
all_cases_time['fips'] = all_cases_time['fips'].astype(float).astype(int)
all_cases_time['fips'] = all_cases_time.fips.map("{:05}".format)

all_cases_time

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,county,state,fips,cases,deaths,population,population_adjusted_cases
date,,,,,,,
2020-01-21,Snohomish,Washington,53061,1,0,822083,0.000001
2020-01-22,Snohomish,Washington,53061,1,0,822083,0.000001
2020-01-23,Snohomish,Washington,53061,1,0,822083,0.000001
2020-01-24,Snohomish,Washington,53061,1,0,822083,0.000001
2020-01-25,Snohomish,Washington,53061,1,0,822083,0.000001
...,...,...,...,...,...,...,...
2020-05-09,Reeves,Texas,48389,1,0,15976,0.000063
2020-05-09,Anderson,Kansas,20003,1,0,7858,0.000127
2020-05-09,Laclede,Missouri,29105,1,0,35723,0.000028


In [0]:
# Helper function to get Y value given start_date and end_date
def get_difference_df(df, start_date, end_date):
  beginning_cases = all_cases_time[start_date]
  end_cases = all_cases_time[end_date]

  fips = list(set(beginning_cases.fips) & set(end_cases.fips)) #only want the cases recorded on both days
  beginning_cases = beginning_cases[beginning_cases.fips.isin(fips)]
  end_cases = end_cases[end_cases.fips.isin(fips)]
  end_cases["end_cases"] = end_cases.cases
  beginning_cases["beginning_cases"] = beginning_cases.cases

  case_difference = beginning_cases.reset_index()
  del case_difference['cases']
  del case_difference['date']

  case_difference['end_cases']= end_cases.reset_index().cases
  case_difference['end_deaths'] = end_cases.reset_index().deaths
  case_difference['case_diff'] = case_difference['end_cases']-case_difference['beginning_cases']
  case_difference['perc_increase'] = case_difference['case_diff']/case_difference['beginning_cases']*100
  case_difference['adj_increase'] = case_difference['case_diff']/case_difference['population']

  return case_difference

In [0]:
START_DATE = '2020-04-08'
END_DATE = '2020-04-15'
case_diff_1 = get_difference_df(all_cases_time, START_DATE, END_DATE)

START_DATE = '2020-04-15'
END_DATE = '2020-04-22'
case_diff_2 = get_difference_df(all_cases_time, START_DATE, END_DATE)

START_DATE = '2020-04-22'
END_DATE = '2020-04-29'
case_diff_3 = get_difference_df(all_cases_time, START_DATE, END_DATE)

In [0]:
case_diff = case_diff_1[['county', 'state', 'fips', 'population']]
case_diff.head()

,county,state,fips,population
0,Snohomish,Washington,53061,822083
1,Cook,Illinois,17031,5150233
2,Orange,California,06059,3175692
3,Maricopa,Arizona,04013,4485414
4,Los Angeles,California,06037,10039107


In [0]:
case_diff['Y1_case_diff'] = float('nan')
case_diff['Y1_perc_increase'] = float('nan')
case_diff['Y1_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  if len(case_diff_1.loc[case_diff_1['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_case_diff'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_perc_increase'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y1_adj_increase'] = case_diff_1.loc[case_diff_1['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,county,state,fips,population,Y1_case_diff,Y1_perc_increase,Y1_adj_increase
0,Snohomish,Washington,53061,822083,293.0,17.296340,0.000356
1,Cook,Illinois,17031,5150233,6786.0,64.505703,0.001318
2,Orange,California,06059,3175692,360.0,35.433071,0.000113
3,Maricopa,Arizona,04013,4485414,587.0,37.652341,0.000131
4,Los Angeles,California,06037,10039107,2966.0,39.389110,0.000295


In [0]:
case_diff['Y2_case_diff'] = float('nan')
case_diff['Y2_perc_increase'] = float('nan')
case_diff['Y2_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  # print(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values)
  if len(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_case_diff'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_perc_increase'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y2_adj_increase'] = case_diff_2.loc[case_diff_2['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,county,state,fips,population,Y1_case_diff,Y1_perc_increase,Y1_adj_increase,Y2_case_diff,Y2_perc_increase,Y2_adj_increase
0,Snohomish,Washington,53061,822083,293.0,17.296340,0.000356,221.0,11.122295,0.000269
1,Cook,Illinois,17031,5150233,6786.0,64.505703,0.001318,7240.0,41.835202,0.001406
2,Orange,California,06059,3175692,360.0,35.433071,0.000113,377.0,27.398256,0.000119
3,Maricopa,Arizona,04013,4485414,587.0,37.652341,0.000131,700.0,32.618826,0.000156
4,Los Angeles,California,06037,10039107,2966.0,39.389110,0.000295,5939.0,56.583460,0.000592


In [0]:
case_diff['Y3_case_diff'] = float('nan')
case_diff['Y3_perc_increase'] = float('nan')
case_diff['Y3_adj_increase'] = float('nan')

for idx, row in case_diff.iterrows():
  # print(case_diff_2.loc[case_diff_2['fips'] == row['fips']]['case_diff'].values)
  if len(case_diff_3.loc[case_diff_3['fips'] == row['fips']]['case_diff'].values) > 0:
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_case_diff'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['case_diff'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_perc_increase'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['perc_increase'].values
    case_diff.loc[case_diff['fips']==row['fips'], 'Y3_adj_increase'] = case_diff_3.loc[case_diff_3['fips'] == row['fips']]['adj_increase'].values

case_diff.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,county,state,fips,population,Y1_case_diff,Y1_perc_increase,Y1_adj_increase,Y2_case_diff,Y2_perc_increase,Y2_adj_increase,Y3_case_diff,Y3_perc_increase,Y3_adj_increase
0,Snohomish,Washington,53061,822083,293.0,17.296340,0.000356,221.0,11.122295,0.000269,208.0,9.420290,0.000253
1,Cook,Illinois,17031,5150233,6786.0,64.505703,0.001318,7240.0,41.835202,0.001406,10334.0,42.100546,0.002007
2,Orange,California,06059,3175692,360.0,35.433071,0.000113,377.0,27.398256,0.000119,571.0,32.572732,0.000180
3,Maricopa,Arizona,04013,4485414,587.0,37.652341,0.000131,700.0,32.618826,0.000156,877.0,30.815179,0.000196
4,Los Angeles,California,06037,10039107,2966.0,39.389110,0.000295,5939.0,56.583460,0.000592,6050.0,36.811682,0.000603


In [0]:
case_diff.to_csv('case_diff_Y_v2.csv')

In [0]:
from google.colab import files
files.download('case_diff_Y_v2.csv')

UNACAST DATA AS T

In [0]:
county_unacast = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/county_unacast.csv')
county_unacast.head()

,Unnamed: 0,date,county_name,county_fips,vistation_grade,travel_distance_grade,total_grade,num_grade
0,0,04/05/20,Bell County,48027,C,C,C,5.0
1,1,04/04/20,Bell County,48027,C,C,C,5.0
2,2,04/05/20,Burleson County,48051,NaN,C,C,5.0
3,3,04/04/20,Burleson County,48051,NaN,C,C,5.0
4,4,04/05/20,Comanche County,48093,NaN,C,C,5.0


In [0]:
county_unacast.loc[county_unacast['county_fips']==53061, 'num_grade'].values.mean()
print(county_unacast['county_fips'])

0       48027
1       48027
2       48051
3       48051
4       48093
        ...  
6435    54083
6436    55033
6437    55033
6438    55083
6439    55083
Name: county_fips, Length: 6440, dtype: int64


In [0]:
unacast_T = case_diff[['county', 'state', 'fips', 'population']]
unacast_T.head()

,county,state,fips,population
0,Snohomish,Washington,53061,822083
1,Cook,Illinois,17031,5150233
2,Orange,California,06059,3175692
3,Maricopa,Arizona,04013,4485414
4,Los Angeles,California,06037,10039107


In [0]:
unacast_T['fips']

0       53061
1       17031
2       06059
3       04013
4       06037
        ...  
2643    46119
2644    46129
2645    54023
2646    54059
2647    56003
Name: fips, Length: 2648, dtype: object

In [0]:
unacast_T['unacast_score'] = 0
for idx, row in unacast_T.iterrows():
  if len(county_unacast.loc[county_unacast['county_fips']==int(row['fips']), 'num_grade'].values) > 0:
    unacast_T.loc[unacast_T['fips']==row['fips'], 'unacast_score'] = county_unacast.loc[county_unacast['county_fips']==int(row['fips']), 'num_grade'].values.mean()

unacast_T.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,county,state,fips,population,unacast_score
0,Snohomish,Washington,53061,822083,6.0
1,Cook,Illinois,17031,5150233,7.5
2,Orange,California,06059,3175692,8.0
3,Maricopa,Arizona,04013,4485414,6.5
4,Los Angeles,California,06037,10039107,7.5


In [0]:
del unacast_T['population']
unacast_T.head()

,county,state,fips,unacast_score
0,Snohomish,Washington,53061,6.0
1,Cook,Illinois,17031,7.5
2,Orange,California,06059,8.0
3,Maricopa,Arizona,04013,6.5
4,Los Angeles,California,06037,7.5


In [0]:
unacast_T.to_csv('unacast_T.csv')
files.download('unacast_T.csv')

Covariates X

1. Demographics: population (total), male/female ratio, age distribution (18 classes; group 1 is 0-5, group 2 is 5-10, ...), race distribution (white, black, native american, asian, hispanic, ...)
2. Socioeconomics: Income, Education, Poverty
3. COVID-19: Current total number of confirmed cases/deaths, tests given, positive/negative test ratio, comorbidities

In [0]:
# Demographics: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/cc-est2018-alldata.pdf
#               cc-est2018-alldata.csv

demographic = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/cc-est2018-alldata.csv', encoding='latin1')
demographic.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,21295,22002,4559,5130,119,139,200,284,29,18,367,429,21633,22391,4704,5306,277,314,300,409,42,37,25875,27386,20709,21485,4512,5091,103,115,194,280,13,9,344,406,21026,21853,4647,5258,251,282,291,398,23,27,694,616,586,517,47,39,16,24,6,4,16,9,23,23,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,1411,1316,362,317,5,3,13,15,1,0,74,62,1479,1368,405,362,23,18,34,28,3,1,1778,1651,1337,1260,356,313,2,2,13,15,0,0,70,61,1402,1312,396,357,19,17,34,28,1,0,88,62,74,56,6,4,3,1,0,0,1,0,4,1,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,1521,1526,399,374,14,8,17,21,1,3,49,58,1570,1583,425,403,27,19,32,42,3,4,1933,1916,1460,1465,398,372,12,2,17,21,0,3,46,53,1506,1517,423,400,25,12,30,39,1,4,68,74,61,61,1,2,2,6,0,0,1,0,3,5,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,1658,1620,431,406,15,12,23,18,4,1,40,62,1694,1681,453,436,29,27,32,37,4,5,2105,2055,1613,1570,421,403,12,9,22,18,3,0,34,55,1643,1624,440,429,24,22,30,36,3,4,66,64,45,50,10,3,3,3,1,0,1,1,6,7,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,1628,1585,502,424,12,7,25,14,4,2,42,45,1664,1624,525,444,23,20,39,31,6,5,2153,2026,1580,1543,495,420,12,5,23,14,1,1,42,43,1616,1580,518,439,23,18,37,30,3,4,60,51,48,42,7,4,0,2,2,0,3,1,0,2,48,44,7,5,0,2,2,1,3,1


In [0]:
# Things to note about processing this
# FIPS = 1000 * STATE + COUNTY
# YEAR: 11 is the most recent (July 2018)
# AGEGRP: 0 is total, 1~18 represent different age group
# Race: WA (White), BA (Black), IA (American + Alsaka Native), AA (Asian), NA (Native Hawaiian + Pacific Islander), H (Hispanic)

# What do we want:
# FIPS, TOTAL_MALE (%), TOTAL_FEMALE (%), AGE DISTRIBUTION (1~18, %), RACE DISTRIBUTION (W, B, I, A, N, H in %)

# 1) Filter by YEAR = 11
recent_demo = demographic.loc[demographic['YEAR']==11]
recent_demo

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
190,50,1,1,Alabama,Autauga County,11,0,55601,26995,28606,20859,21801,5153,5762,121,146,301,380,33,29,528,488,21334,22246,5442,6021,277,294,453,509,58,59,26135,27817,20157,21159,5074,5681,105,138,297,373,20,23,482,443,20596,21566,5334,5919,241,261,434,492,36,47,860,789,702,642,79,81,16,8,4,7,13,6,46,45,738,680,108,102,36,33,19,17,22,12
191,50,1,1,Alabama,Autauga County,11,1,3364,1728,1636,1240,1167,354,359,5,6,29,19,3,4,97,81,1333,1242,426,420,14,13,51,35,7,8,1638,1562,1169,1109,348,357,3,5,29,17,1,3,88,71,1254,1176,414,415,10,6,48,31,2,5,90,74,71,58,6,2,2,1,0,2,2,1,9,10,79,66,12,5,4,7,3,4,5,3
192,50,1,1,Alabama,Autauga County,11,2,3423,1779,1644,1296,1194,361,340,8,9,14,23,2,2,98,76,1388,1266,433,396,19,14,38,40,5,4,1687,1580,1215,1134,361,338,6,9,13,23,2,2,90,74,1302,1204,428,394,14,12,34,40,2,4,92,64,81,60,0,2,2,0,1,0,0,0,8,2,86,62,5,2,5,2,4,0,3,0
193,50,1,1,Alabama,Autauga County,11,3,3882,1977,1905,1467,1411,432,391,4,7,15,31,4,4,55,61,1516,1470,472,429,11,17,31,47,6,5,1891,1829,1398,1349,426,384,4,7,14,30,1,3,48,56,1442,1403,460,420,8,13,29,45,3,4,86,76,69,62,6,7,0,0,1,1,3,1,7,5,74,67,12,9,3,4,2,2,3,1
194,50,1,1,Alabama,Autauga County,11,4,3755,1913,1842,1395,1385,426,383,10,4,26,11,4,3,52,56,1442,1435,462,421,19,13,39,30,6,8,1845,1770,1340,1324,421,382,9,4,26,11,3,2,46,47,1381,1367,452,414,14,9,39,28,5,5,68,72,55,61,5,1,1,0,0,0,1,1,6,9,61,68,10,7,5,4,0,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656673,50,56,45,Wyoming,Weston County,11,14,473,267,206,254,195,0,0,2,0,9,8,0,0,2,3,256,198,0,1,4,1,9,9,0,0,263,201,250,190,0,0,2,0,9,8,0,0,2,3,252,193,0,1,4,1,9,9,0,0,4,5,4,5,0,0,0,0,0,0,0,0,0,0,4,5,0,0,0,0,0,0,0,0
656674,50,56,45,Wyoming,Weston County,11,15,336,168,168,161,164,0,0,1,2,2,1,0,0,4,1,165,165,1,0,4,3,2,1,0,0,164,167,159,163,0,0,1,2,0,1,0,0,4,1,163,164,1,0,4,3,0,1,0,0,4,1,2,1,0,0,0,0,2,0,0,0,0,0,2,1,0,0,0,0,2,0,0,0
656675,50,56,45,Wyoming,Weston County,11,16,224,104,120,102,119,0,0,0,1,0,0,0,0,2,0,104,119,0,0,2,1,0,0,0,0,103,119,101,118,0,0,0,1,0,0,0,0,2,0,103,118,0,0,2,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
656676,50,56,45,Wyoming,Weston County,11,17,196,88,108,86,106,0,0,1,2,0,0,0,0,1,0,87,106,0,0,2,2,0,0,0,0,86,107,85,105,0,0,0,2,0,0,0,0,1,0,86,105,0,0,1,2,0,0,0,0,2,1,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0


In [0]:
# 2) Create a FIPS column
recent_demo['fips'] = recent_demo['STATE'] * 1000 + recent_demo['COUNTY']
recent_demo['fips'] = recent_demo['fips'].astype(float).astype(int)
recent_demo['fips'] = recent_demo.fips.map("{:05}".format)
#all_cases_time['fips'] = all_cases_time['fips'].astype(float).astype(int)
#all_cases_time['fips'] = all_cases_time.fips.map("{:05}".format)
recent_demo.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,...,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,fips
190,50,1,1,Alabama,Autauga County,11,0,55601,26995,28606,20859,21801,5153,5762,121,146,301,380,33,29,528,488,21334,22246,5442,6021,277,294,453,509,58,59,26135,27817,20157,21159,5074,5681,105,138,...,373,20,23,482,443,20596,21566,5334,5919,241,261,434,492,36,47,860,789,702,642,79,81,16,8,4,7,13,6,46,45,738,680,108,102,36,33,19,17,22,12,01001
191,50,1,1,Alabama,Autauga County,11,1,3364,1728,1636,1240,1167,354,359,5,6,29,19,3,4,97,81,1333,1242,426,420,14,13,51,35,7,8,1638,1562,1169,1109,348,357,3,5,...,17,1,3,88,71,1254,1176,414,415,10,6,48,31,2,5,90,74,71,58,6,2,2,1,0,2,2,1,9,10,79,66,12,5,4,7,3,4,5,3,01001
192,50,1,1,Alabama,Autauga County,11,2,3423,1779,1644,1296,1194,361,340,8,9,14,23,2,2,98,76,1388,1266,433,396,19,14,38,40,5,4,1687,1580,1215,1134,361,338,6,9,...,23,2,2,90,74,1302,1204,428,394,14,12,34,40,2,4,92,64,81,60,0,2,2,0,1,0,0,0,8,2,86,62,5,2,5,2,4,0,3,0,01001
193,50,1,1,Alabama,Autauga County,11,3,3882,1977,1905,1467,1411,432,391,4,7,15,31,4,4,55,61,1516,1470,472,429,11,17,31,47,6,5,1891,1829,1398,1349,426,384,4,7,...,30,1,3,48,56,1442,1403,460,420,8,13,29,45,3,4,86,76,69,62,6,7,0,0,1,1,3,1,7,5,74,67,12,9,3,4,2,2,3,1,01001
194,50,1,1,Alabama,Autauga County,11,4,3755,1913,1842,1395,1385,426,383,10,4,26,11,4,3,52,56,1442,1435,462,421,19,13,39,30,6,8,1845,1770,1340,1324,421,382,9,4,...,11,3,2,46,47,1381,1367,452,414,14,9,39,28,5,5,68,72,55,61,5,1,1,0,0,0,1,1,6,9,61,68,10,7,5,4,0,2,1,3,01001


In [0]:
# 3) Create a new df with fips, stname, ctyname
df = recent_demo[['fips', 'STNAME', 'CTYNAME']].drop_duplicates()
df.head()

,fips,STNAME,CTYNAME
190,01001,Alabama,Autauga County
399,01003,Alabama,Baldwin County
608,01005,Alabama,Barbour County
817,01007,Alabama,Bibb County
1026,01009,Alabama,Blount County


In [0]:
temp = recent_demo.loc[recent_demo['fips']=='01001']
temp.loc[temp['AGEGRP']==0]['TOT_POP'].values[0]

55601

In [0]:
# 4) Add relevant data
df['male'] = 0
df['female'] = 0

for i in range(1, 19, 1):
  age_grp = 'age_' + str(i)
  df[age_grp] = 0

df['white'] = 0
df['black'] = 0
df['native'] = 0
df['pacific'] = 0
df['asian'] = 0
df['hispanic'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  demo = recent_demo.loc[recent_demo['fips']==fips]
  total_pop = demo.loc[demo['AGEGRP']==0]['TOT_POP'].values[0]
  total_male = demo.loc[demo['AGEGRP']==0]['TOT_MALE'].values[0]
  total_female = demo.loc[demo['AGEGRP']==0]['TOT_FEMALE'].values[0]

  age = []
  for i in range(1, 19, 1):
    age.append(demo.loc[demo['AGEGRP']==i]['TOT_POP'].values[0])

  white = demo.loc[demo['AGEGRP']==0]['WA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['WA_FEMALE'].values[0]
  black = demo.loc[demo['AGEGRP']==0]['BA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['BA_FEMALE'].values[0]
  native = demo.loc[demo['AGEGRP']==0]['IA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['IA_FEMALE'].values[0]
  pacific = demo.loc[demo['AGEGRP']==0]['NA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['NA_FEMALE'].values[0]
  asian = demo.loc[demo['AGEGRP']==0]['AA_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['AA_FEMALE'].values[0]
  hispanic = demo.loc[demo['AGEGRP']==0]['H_MALE'].values[0] + demo.loc[demo['AGEGRP']==0]['H_FEMALE'].values[0]

  df.loc[df['fips']==fips, 'male'] = float(total_male) / float(total_pop)
  df.loc[df['fips']==fips, 'female'] = float(total_female) / float(total_pop)

  for i in range(1, 19, 1):
    agegrp = 'age_' + str(i)
    df.loc[df['fips']==fips, agegrp] = float(age[i-1]) / float(total_pop)

  df.loc[df['fips']==fips, 'white'] = float(white) / float(total_pop)
  df.loc[df['fips']==fips, 'black'] = float(black) / float(total_pop)
  df.loc[df['fips']==fips, 'native'] = float(native) / float(total_pop)
  df.loc[df['fips']==fips, 'pacific'] = float(pacific) / float(total_pop)
  df.loc[df['fips']==fips, 'asian'] = float(asian) / float(total_pop)
  df.loc[df['fips']==fips, 'hispanic'] = float(hispanic) / float(total_pop)

  # df.head()


In [0]:
df['total_pop'] = 0
for idx, row in df.iterrows():
  fips = row['fips']
  demo = recent_demo.loc[recent_demo['fips']==fips]
  total_pop = demo.loc[demo['AGEGRP']==0]['TOT_POP'].values[0]
  df.loc[df['fips']==fips, 'total_pop'] = total_pop

df.head()

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop
190,01001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601
399,01003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022
608,01005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881
817,01007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400
1026,01009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840


In [0]:
from google.colab import files
df.to_csv('demographics_X.csv')
files.download('demographics_X.csv')

In [0]:
df.head()

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop
190,01001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601
399,01003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022
608,01005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881
817,01007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400
1026,01009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840


In [0]:
edu_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/Education.xls')
poverty_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/PovertyEstimates.xls')
income_df = pd.read_excel('/content/drive/My Drive/COVID19 Data Exploration /Data/Unemployment.xls')

In [0]:
edu_df = edu_df[4:]
edu_df = edu_df[['Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2018', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46']]
# edu_df['Unnamed: 46']
edu_df.head()

,"Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2018",Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
4,00000,12.3,27.1,29,31.5
5,01000,14.2,30.9,29.9,24.9
6,01001,11.3,32.6,28.4,27.7
7,01003,9.7,27.6,31.3,31.3
8,01005,27,35.7,25.1,12.2


In [0]:
edu_df = edu_df.rename(columns = {'Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2018':'FIPS'})
edu_df = edu_df.rename(columns = {'Unnamed: 43':'LessHighSchool', 'Unnamed: 44':'OnlyHighSchool', 'Unnamed: 45':'SomeBachelor', 'Unnamed: 46':'Bachelor'})
edu_df.head()

,FIPS,LessHighSchool,OnlyHighSchool,SomeBachelor,Bachelor
4,00000,12.3,27.1,29,31.5
5,01000,14.2,30.9,29.9,24.9
6,01001,11.3,32.6,28.4,27.7
7,01003,9.7,27.6,31.3,31.3
8,01005,27,35.7,25.1,12.2


In [0]:
poverty_df = poverty_df[4:]
poverty_df = poverty_df[['Poverty estimates for the U.S., States, and counties, 2018', 'Unnamed: 7']]

poverty_df.head()

,"Poverty estimates for the U.S., States, and counties, 2018",Unnamed: 7
4,00000,41852315
5,01000,801758
6,01001,7587
7,01003,21069
8,01005,6788


In [0]:
poverty_df = poverty_df.rename(columns={'Poverty estimates for the U.S., States, and counties, 2018':'FIPS', 'Unnamed: 7':'Poverty'})
poverty_df.head()

,FIPS,Poverty
4,00000,41852315
5,01000,801758
6,01001,7587
7,01003,21069
8,01005,6788


In [0]:
income_df = income_df[4:]
income_df = income_df[['Unemployment and median household income for the U.S., States, and counties, 2007-18', 'Unnamed: 53', 'Unnamed: 54']]

income_df.head()

,"Unemployment and median household income for the U.S., States, and counties, 2007-18",Unnamed: 53,Unnamed: 54
4,00000,3.9,61937
5,01000,3.9,49881
6,01001,3.6,59338
7,01003,3.6,57588
8,01005,5.2,34382


In [0]:
income_df = income_df.rename(columns={'Unemployment and median household income for the U.S., States, and counties, 2007-18':'FIPS', \
                                      'Unnamed: 53':'UnemploymentRate', 'Unnamed: 54':'MedianIncome'})
income_df.head()

,FIPS,UnemploymentRate,MedianIncome
4,00000,3.9,61937
5,01000,3.9,49881
6,01001,3.6,59338
7,01003,3.6,57588
8,01005,5.2,34382


In [0]:
df.head()

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop
190,01001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601
399,01003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022
608,01005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881
817,01007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400
1026,01009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840


In [0]:
df['Education_LessHighSchool'] = 0
df['Education_OnlyHighSchool'] = 0
df['Education_SomeBachelor'] = 0
df['Education_Bachelor'] = 0

df['Poverty'] = 0
df['Unemployment'] = 0
df['MedianIncome'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  
  edu = edu_df.loc[edu_df['FIPS']==fips]
  if len(edu) > 0:
    df.loc[df['fips']==fips, 'Education_LessHighSchool'] = float(edu['LessHighSchool'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_OnlyHighSchool'] = float(edu['OnlyHighSchool'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_SomeBachelor'] = float(edu['SomeBachelor'].values[0]) / 100.
    df.loc[df['fips']==fips, 'Education_Bachelor'] = float(edu['Bachelor'].values[0]) / 100.
  
  if len(poverty_df.loc[poverty_df['FIPS']==fips]) > 0:
    df.loc[df['fips']==fips, 'Poverty'] = float(poverty_df.loc[poverty_df['FIPS']==fips]['Poverty'].values[0]) / float(row['total_pop'])

  if len(income_df.loc[income_df['FIPS']==fips]) > 0:
    df.loc[df['fips']==fips, 'Unemployment'] = float(income_df.loc[income_df['FIPS']==fips]['UnemploymentRate'].values[0]) / 100. 
    df.loc[df['fips']==fips, 'MedianIncome'] = income_df.loc[income_df['FIPS']==fips]['MedianIncome'].values[0]

df.head()



,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop,Education_LessHighSchool,Education_OnlyHighSchool,Education_SomeBachelor,Education_Bachelor,Poverty,Unemployment,MedianIncome
190,01001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601,0.113,0.326,0.284,0.277,0.136454,0.036,59338
399,01003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022,0.097,0.276,0.313,0.313,0.096637,0.036,57588
608,01005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881,0.270,0.357,0.251,0.122,0.272819,0.052,34382
817,01007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400,0.168,0.473,0.244,0.115,0.196429,0.040,46064
1026,01009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840,0.198,0.340,0.335,0.126,0.130135,0.035,50412


In [0]:
df.to_csv('demographics_socioecon_X.csv')
files.download('demographics_socioecon_X.csv')

Combining demographics + socioecon + current COVID status + politics

In [0]:
df = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/demographics_socioecon_X.csv')
del df['Unnamed: 0']
df.head()

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop,Education_LessHighSchool,Education_OnlyHighSchool,Education_SomeBachelor,Education_Bachelor,Poverty,Unemployment,MedianIncome
0,1001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601,0.113,0.326,0.284,0.277,0.136454,0.036,59338
1,1003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022,0.097,0.276,0.313,0.313,0.096637,0.036,57588
2,1005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881,0.270,0.357,0.251,0.122,0.272819,0.052,34382
3,1007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400,0.168,0.473,0.244,0.115,0.196429,0.040,46064
4,1009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840,0.198,0.340,0.335,0.126,0.130135,0.035,50412


In [0]:
pop_density_per_county = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/associated-data/master/us_census_data/us_census_2018_population_estimates_counties.csv')
pop_density_per_county.head()

,state,county,state_name,geo_id,population,pop_density
0,AL,Autauga,Alabama,1001,55601,93.534505
1,AL,Baldwin,Alabama,1003,218022,137.138546
2,AL,Barbour,Alabama,1005,24881,28.114060
3,AL,Bibb,Alabama,1007,22400,35.986169
4,AL,Blount,Alabama,1009,57840,89.697955


In [0]:
state_tests = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv')
state_tests.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,hash,dateChecked,death,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200510,AK,379.0,26985.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,324.0,C,5/10/2020 14:00,c326467537b8aa4035125d086454fe043009b85a,2020-05-10T20:00:00Z,10.0,NaN,27364,27364,27364,2,0.0,0.0,914.0,1.0,915.0
1,20200510,AL,9777.0,117644.0,NaN,NaN,1240.0,NaN,460.0,NaN,274.0,NaN,B,5/10/2020 00:00,b784245157e15325dbb3d93142f0dd3e69664809,2020-05-10T20:00:00Z,393.0,1240.0,127421,127421,127421,1,5.0,12.0,1717.0,210.0,1927.0
2,20200510,AR,3747.0,61781.0,NaN,64.0,471.0,NaN,NaN,14.0,96.0,2968.0,B,5/10/2020 10:15,38a9c539abfb2c4f9d79d7cf330d67eada846df5,2020-05-10T20:00:00Z,88.0,471.0,65528,65528,65528,5,0.0,0.0,0.0,0.0,0.0
3,20200510,AS,0.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,5/3/2020 00:00,d66ec1502bd760d9b7e029ada9baca3d2c30db82,2020-05-10T20:00:00Z,0.0,NaN,83,83,83,60,0.0,0.0,0.0,0.0,0.0
4,20200510,AZ,11119.0,126620.0,NaN,713.0,1528.0,300.0,NaN,195.0,NaN,2775.0,A+,5/10/2020 00:00,74a1a39b696dae2b812416173663517bf861901e,2020-05-10T20:00:00Z,536.0,1528.0,137739,137739,137739,4,4.0,14.0,8640.0,159.0,8799.0


In [0]:
election_data = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv')
election_data.head()

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [0]:
election_data.loc[election_data['combined_fips']//1000 == 2]

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060
5,5,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2068
6,6,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2070
7,7,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2090
8,8,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2100
9,9,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2105


In [0]:
df['pop_density'] = 0
df['total_test'] = 0
df['pos_test_rate'] = 0
df['dem_support'] = 0
df['gop_support'] = 0

for idx, row in df.iterrows():
  fips = row['fips']
  state_fips = int(fips / 1000)
  state_pop = df.loc[df['fips']//1000==state_fips]['total_pop'].values.sum()

  if len( pop_density_per_county.loc[pop_density_per_county['geo_id']==fips]) > 0:
    df.loc[df['fips']==fips, 'pop_density'] = pop_density_per_county.loc[pop_density_per_county['geo_id']==fips]['pop_density'].values[0]

  if len(state_tests.loc[state_tests['fips']==state_fips]) > 0:
    pos_test = state_tests.loc[state_tests['fips']==state_fips]['positive'].values[0]
    neg_test = state_tests.loc[state_tests['fips']==state_fips]['negative'].values[0]
    df.loc[df['fips']==fips, 'total_test'] = (float(pos_test) + float(neg_test)) / float(state_pop)
    df.loc[df['fips']==fips, 'pos_test_rate'] = float(pos_test) / (float(pos_test) + float(neg_test))
  # print(fips)
  if len(election_data.loc[election_data['combined_fips']==fips]) > 0:
    df.loc[df['fips']==fips, 'dem_support'] = election_data.loc[election_data['combined_fips']==fips]['per_dem'].values[0]
    df.loc[df['fips']==fips, 'gop_support'] = election_data.loc[election_data['combined_fips']==fips]['per_gop'].values[0]

df.head()

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop,Education_LessHighSchool,Education_OnlyHighSchool,Education_SomeBachelor,Education_Bachelor,Poverty,Unemployment,MedianIncome,pop_density,total_test,pos_test_rate,dem_support,gop_support
0,1001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601,0.113,0.326,0.284,0.277,0.136454,0.036,59338,93.534505,0.026069,0.07673,0.239569,0.734358
1,1003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022,0.097,0.276,0.313,0.313,0.096637,0.036,57588,137.138546,0.026069,0.07673,0.195653,0.773515
2,1005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881,0.270,0.357,0.251,0.122,0.272819,0.052,34382,28.114060,0.026069,0.07673,0.466603,0.522714
3,1007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400,0.168,0.473,0.244,0.115,0.196429,0.040,46064,35.986169,0.026069,0.07673,0.214220,0.769662
4,1009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840,0.198,0.340,0.335,0.126,0.130135,0.035,50412,89.697955,0.026069,0.07673,0.084699,0.898519


In [0]:
# Now, add covid related data
# 1) total case until March 28 (a week before Apr 4)
# 2) increase during March 23-28

START_DATE = '2020-03-25'
END_DATE = '2020-04-01'
case_diff_X = get_difference_df(all_cases_time, START_DATE, END_DATE)
case_diff_X.head()

,county,state,fips,deaths,population,population_adjusted_cases,beginning_cases,end_cases,end_deaths,case_diff,perc_increase,adj_increase
0,Snohomish,Washington,53061,15,822083,0.000770,633,1304,39,671,106.003160,0.000816
1,Cook,Illinois,17031,13,5150233,0.000275,1418,5152,95,3734,263.328632,0.000725
2,Orange,California,06059,1,3175692,0.000059,187,606,10,419,224.064171,0.000132
3,Maricopa,Arizona,04013,2,4485414,0.000056,251,871,8,620,247.011952,0.000138
4,Los Angeles,California,06037,13,10039107,0.000080,799,3518,65,2719,340.300375,0.000271


In [0]:
df['fips'] = df['fips'].astype(float).astype(int)
df['fips'] = df.fips.map("{:05}".format)

In [0]:
data_from_end = all_cases_time['2020-04-01']

In [0]:
df['cases'] = 0
df['deaths'] = 0
df['perc_increase'] = 0
df['adj_increase'] = 0

for idx, row in df.iterrows():
  if len(data_from_end.loc[data_from_end['fips']==row['fips']]) > 0:
    df.loc[df['fips']==row['fips'], 'cases'] = data_from_end.loc[data_from_end['fips']==row['fips']]['cases'].values[0]
    df.loc[df['fips']==row['fips'], 'deaths'] = data_from_end.loc[data_from_end['fips']==row['fips']]['deaths'].values[0]
  if len(case_diff_X.loc[case_diff_X['fips']==row['fips']]) > 0:
    df.loc[df['fips']==row['fips'], 'perc_increase'] = case_diff_X.loc[case_diff_X['fips']==row['fips']]['perc_increase'].values[0]
    df.loc[df['fips']==row['fips'], 'adj_increase'] = case_diff_X.loc[case_diff_X['fips']==row['fips']]['adj_increase'].values[0]

In [0]:
df.head(30)

,fips,STNAME,CTYNAME,male,female,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16,age_17,age_18,white,black,native,pacific,asian,hispanic,total_pop,Education_LessHighSchool,Education_OnlyHighSchool,Education_SomeBachelor,Education_Bachelor,Poverty,Unemployment,MedianIncome,pop_density,total_test,pos_test_rate,dem_support,gop_support,cases,deaths,perc_increase,adj_increase
0,01001,Alabama,Autauga County,0.485513,0.514487,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.767252,0.196309,0.004802,0.001115,0.012248,0.029658,55601,0.113,0.326,0.284,0.277,0.136454,0.036,59338,93.534505,0.026069,0.07673,0.239569,0.734358,10,0,150.000000,0.000107
1,01003,Alabama,Baldwin County,0.484616,0.515384,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.872852,0.089404,0.007724,0.000670,0.011503,0.046468,218022,0.097,0.276,0.313,0.313,0.096637,0.036,57588,137.138546,0.026069,0.07673,0.195653,0.773515,23,1,475.000000,0.000085
2,01005,Alabama,Barbour County,0.527832,0.472168,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.490696,0.483984,0.006591,0.001849,0.004542,0.042764,24881,0.270,0.357,0.251,0.122,0.272819,0.052,34382,28.114060,0.026069,0.07673,0.466603,0.522714,0,0,0.000000,0.000000
3,01007,Alabama,Bibb County,0.532188,0.467813,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.768348,0.212946,0.004375,0.001161,0.002366,0.026250,22400,0.168,0.473,0.244,0.115,0.196429,0.040,46064,35.986169,0.026069,0.07673,0.214220,0.769662,3,0,0.000000,0.000000
4,01009,Alabama,Blount County,0.492739,0.507261,0.059820,0.062846,0.067462,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.958783,0.016425,0.006535,0.001210,0.003198,0.095712,57840,0.198,0.340,0.335,0.126,0.130135,0.035,50412,89.697955,0.026069,0.07673,0.084699,0.898519,5,0,400.000000,0.000069
5,01011,Alabama,Bullock County,0.545177,0.454823,0.059183,0.058986,0.059282,0.052180,0.061748,0.075853,0.074670,0.067962,0.061156,0.064510,0.059578,0.074176,0.066877,0.053364,0.040146,0.029197,0.022983,0.018150,0.262478,0.707043,0.008384,0.007497,0.001874,0.079601,10138,0.248,0.397,0.223,0.133,0.356086,0.047,29267,16.277951,0.026069,0.07673,0.750904,0.242289,3,0,0.000000,0.000000
6,01013,Alabama,Butler County,0.465701,0.534299,0.056860,0.062602,0.065447,0.059604,0.055386,0.060772,0.055030,0.063720,0.057724,0.060061,0.057470,0.069106,0.073222,0.063110,0.048679,0.035671,0.025813,0.029726,0.522510,0.449136,0.003760,0.000508,0.013161,0.015091,19680,0.154,0.439,0.246,0.161,0.240396,0.048,37365,25.333449,0.026069,0.07673,0.427864,0.563155,1,0,0.000000,0.000000
7,01015,Alabama,Calhoun County,0.480534,0.519466,0.057579,0.057413,0.062532,0.065551,0.063757,0.071554,0.060222,0.061106,0.058306,0.061815,0.063346,0.070758,0.068885,0.059233,0.046440,0.032132,0.020897,0.018473,0.752785,0.211355,0.005399,0.001103,0.009643,0.039107,114277,0.159,0.324,0.337,0.180,0.190056,0.047,45400,188.617140,0.026069,0.07673,0.278559,0.692397,11,0,450.000000,0.000079
8,01017,Alabama,Chambers County,0.478745,0.521255,0.057385,0.057712,0.057296,0.056106,0.059705,0.070593,0.055273,0.055808,0.056493,0.064822,0.066339,0.075294,0.071962,0.061669,0.052566,0.035847,0.022847,0.022282,0.570638,0.400000,0.003064,0.000446,0.013268,0.025614,33615,0.186,0.384,0.297,0.132,0.183876,0.039,39917,56.347976,0.026069,0.07673,0.418276,0.566338,45,4,350.000000,0.001053
9,01019,Alabama,Cherokee County,0.495390,0.504610,0.046328,0.051782,0.056930,0.057583,0.050092,0.059465,0.0463

In [0]:
from google.colab import files
df.to_csv('all_X_ver2.csv')
files.download('all_X_ver2.csv')